# Checkings

In [56]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from glob import glob

numpy2ri.activate()  # enables automatic conversion between R and numpy objects

# Import the R function for reading RDS files.
readRDS = ro.r['readRDS']

def load_rds_as_sparse(file_path):
    """
    Load an .rds file that contains a dgCMatrix and return:
      - a scipy.sparse.csc_matrix for the gene expression data,
      - the gene symbols (row names),
      - the cell metadata as a pandas DataFrame.
      
    This function assumes that:
      - The dgCMatrix has slots "i", "p", "x", "Dim", and "Dimnames".
      - The cell metadata is stored in the slot "metadata".
    """
    r_obj = readRDS(file_path)
    
    # Access the S4 slots using ro.r['slot']
    i_slot = np.array(ro.r['slot'](r_obj, 'i'))
    p_slot = np.array(ro.r['slot'](r_obj, 'p'))
    x_slot = np.array(ro.r['slot'](r_obj, 'x'))
    dim = np.array(ro.r['slot'](r_obj, 'Dim'))
    sparse_mat = csc_matrix((x_slot, i_slot, p_slot), shape=(dim[0], dim[1]))
    
    # Get gene names from the first element of the Dimnames slot
    dimnames = ro.r['slot'](r_obj, 'Dimnames')
    genes = list(dimnames[0])
    
    # Convert the metadata (cell info) to a pandas DataFrame.
    meta_r = ro.r['slot'](r_obj, 'metadata')
    metadata = pd.DataFrame(np.array(meta_r))
    
    return sparse_mat, genes, metadata, x_slot

sparse, genes, metadata, x = load_rds_as_sparse(r"D:\Macaque\cellFiles\total_gene_T25.rds")

In [19]:
# print(metadata)
# print(x)
nnz_count = len(sparse.indices)
print(nnz_count)

108292190


In [11]:
# Load the region lookup CSV as before.
region_lookup = pd.read_csv('global_region_id_to_atlas_number.csv')
region_lookup_dict = pd.Series(region_lookup.atlas_number.values, index=region_lookup.global_region_id).to_dict()
print(region_lookup_dict)
unique_atlas = sorted(region_lookup.atlas_number.unique())
n_atlas = len(unique_atlas)
print(unique_atlas)

{100: 348, 101: 348, 102: 348, 103: 348, 104: 348, 105: 348, 111: 312, 110: 312, 109: 312, 108: 312, 107: 312, 106: 312, 112: 345, 113: 345, 114: 345, 115: 345, 116: 345, 117: 345, 118: 327, 119: 327, 120: 327, 121: 327, 122: 327, 123: 327, 124: 301, 125: 301, 126: 301, 129: 301, 128: 301, 127: 301, 130: 412, 131: 412, 132: 412, 133: 412, 134: 412, 135: 412, 141: 431, 140: 431, 139: 431, 138: 431, 137: 431, 136: 431, 142: 353, 143: 353, 144: 353, 145: 353, 146: 353, 147: 353, 153: 448, 152: 448, 151: 448, 150: 448, 149: 448, 148: 448, 159: 405, 158: 405, 157: 405, 156: 405, 155: 405, 154: 405, 160: 367, 161: 367, 162: 367, 163: 367, 164: 367, 165: 306, 166: 306, 167: 306, 168: 306, 169: 306, 174: 316, 173: 316, 172: 316, 171: 316, 170: 316, 175: 363, 176: 363, 177: 363, 178: 363, 179: 363, 180: 486, 181: 486, 182: 486, 183: 486, 184: 486, 185: 303, 186: 303, 187: 303, 188: 303, 189: 303, 190: 325, 191: 325, 192: 325, 193: 325, 194: 325, 195: 325, 201: 362, 200: 362, 199: 362, 198: 362,

In [ ]:
# Load the per-slide cell type mapping from pickle.
with open("CellID_Per_Slide_Mapping.pkl", "rb") as f:
    slide_lookup = pickle.load(f)
all_celltypes = set()
for key, df in slide_lookup.items():
    all_celltypes.update(df['celltype_index'].unique())
all_celltype = sorted(all_celltypes)
n_celltype = len(all_celltype)
print(all_celltype)
print(n_celltype)

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), np.int64(69), np.int64(70), np.int64(71), n

In [20]:
# Create mappings from atlas number / celltype to indices in aggregated arrays.
atlas_to_index = {atlas: idx for idx, atlas in enumerate(unique_atlas)}
celltype_to_index = {ct: idx for idx, ct in enumerate(all_celltype)}
print(atlas_to_index)
print(celltype_to_index)

{np.int64(300): 0, np.int64(301): 1, np.int64(303): 2, np.int64(305): 3, np.int64(306): 4, np.int64(308): 5, np.int64(309): 6, np.int64(312): 7, np.int64(313): 8, np.int64(314): 9, np.int64(315): 10, np.int64(316): 11, np.int64(318): 12, np.int64(321): 13, np.int64(322): 14, np.int64(323): 15, np.int64(325): 16, np.int64(326): 17, np.int64(327): 18, np.int64(328): 19, np.int64(329): 20, np.int64(330): 21, np.int64(332): 22, np.int64(333): 23, np.int64(335): 24, np.int64(336): 25, np.int64(337): 26, np.int64(341): 27, np.int64(342): 28, np.int64(343): 29, np.int64(344): 30, np.int64(345): 31, np.int64(347): 32, np.int64(348): 33, np.int64(349): 34, np.int64(350): 35, np.int64(351): 36, np.int64(353): 37, np.int64(354): 38, np.int64(355): 39, np.int64(357): 40, np.int64(358): 41, np.int64(359): 42, np.int64(360): 43, np.int64(361): 44, np.int64(362): 45, np.int64(363): 46, np.int64(364): 47, np.int64(365): 48, np.int64(366): 49, np.int64(367): 50, np.int64(369): 51, np.int64(370): 52, np

In [21]:
all_genes = pd.read_csv(r"D:\Macaque\macaque_genes.csv")
all_genes = all_genes['gene_name'].to_list()
n_genes = len(all_genes)
gene_to_global_idx = {gene: idx for idx, gene in enumerate(all_genes)}
print(gene_to_global_idx)

{'A2M': 0, 'A2ML1': 1, 'A3GALT2': 2, 'A4GALT': 3, 'AAAS': 4, 'AACS': 5, 'AADAC': 6, 'AADACL2': 7, 'AADACL3': 8, 'AADACL4': 9, 'AADAT': 10, 'AAGAB': 11, 'AAK1': 12, 'AAMDC': 13, 'AAMP': 14, 'AANAT': 15, 'AAR2': 16, 'AARD': 17, 'AARS': 18, 'AARS2': 19, 'AASDH': 20, 'AASDHPPT': 21, 'AASS': 22, 'AATF': 23, 'AATK': 24, 'ABAT': 25, 'ABCA1': 26, 'ABCA10': 27, 'ABCA12': 28, 'ABCA13': 29, 'ABCA2': 30, 'ABCA3': 31, 'ABCA4': 32, 'ABCA5': 33, 'ABCA6': 34, 'ABCA7': 35, 'ABCA8': 36, 'ABCA9': 37, 'ABCB1': 38, 'ABCB10': 39, 'ABCB11': 40, 'ABCB5': 41, 'ABCB6': 42, 'ABCB7': 43, 'ABCB8': 44, 'ABCB9': 45, 'ABCC1': 46, 'ABCC10': 47, 'ABCC11': 48, 'ABCC12': 49, 'ABCC2': 50, 'ABCC3': 51, 'ABCC4': 52, 'ABCC5': 53, 'ABCC6': 54, 'ABCC8': 55, 'ABCC9': 56, 'ABCD1': 57, 'ABCD2': 58, 'ABCD3': 59, 'ABCD4': 60, 'ABCE1': 61, 'ABCF1': 62, 'ABCF2': 63, 'ABCF3': 64, 'ABCG1': 65, 'ABCG2': 66, 'ABCG4': 67, 'ABCG5': 68, 'ABCG8': 69, 'ABHD1': 70, 'ABHD10': 71, 'ABHD11': 72, 'ABHD12': 73, 'ABHD12B': 74, 'ABHD13': 75, 'ABHD14B

In [22]:
print(n_genes, n_atlas, n_celltype)

15926 141 258


In [23]:
file_name = os.path.basename(r"D:\Macaque\cellFiles\total_gene_T25.rds")
match = re.search(r"T\d+", file_name)
if not match:
    print(f"Could not extract slide key from {file_name}. Skipping.")
slide_key = match.group(0)
print(slide_key)

T25


In [48]:
file_mapping_df = slide_lookup[slide_key]
# Create a dictionary mapping from cell_id to celltype_index for this slide.
file_cell_lookup = dict(zip(file_mapping_df['cell_id'], file_mapping_df['celltype_index']))
print('Input cell_Id 5, output:')
print(file_cell_lookup[5])

Input cell_Id 5, output:
56


In [57]:
metadata_T = metadata.transpose()
metadata_T.columns = ['cell_id', 'gene_area','x','y','ry','rx']
metadata = metadata_T[['cell_id', 'gene_area',]]
print(metadata)

          cell_id  gene_area
0             4.0      857.0
1             5.0      383.0
2            11.0      826.0
3            12.0      849.0
4            15.0      474.0
...           ...        ...
387090  1009724.0      472.0
387091  1009727.0      472.0
387092  1009735.0      250.0
387093  1009738.0      102.0
387094  1009748.0      474.0

[387095 rows x 2 columns]


In [58]:
# Map each cell's cell_id to its celltype_index using the per-file mapping.
metadata['celltype_index'] = metadata['cell_id'].map(file_cell_lookup)

# Map each cell's 'gene_area' (global_region_id) to its atlas_number using region_lookup.
metadata['atlas_number'] = metadata['gene_area'].map(region_lookup_dict)

print(metadata.shape)
metadata = metadata.dropna(subset=['celltype_index', 'atlas_number'])
print(metadata.shape)
print(metadata)


(387095, 4)
(387095, 4)
          cell_id  gene_area  celltype_index  atlas_number
0             4.0      857.0              20           420
1             5.0      383.0              56           409
2            11.0      826.0              55           458
3            12.0      849.0              10           390
4            15.0      474.0              71           359
...           ...        ...             ...           ...
387090  1009724.0      472.0             100           359
387091  1009727.0      472.0              30           359
387092  1009735.0      250.0              34           482
387093  1009738.0      102.0              17           348
387094  1009748.0      474.0              44           359

[387095 rows x 4 columns]


C:\Users\fjpgr\AppData\Local\Temp\ipykernel_19964\4051551321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['celltype_index'] = metadata['cell_id'].map(file_cell_lookup)
C:\Users\fjpgr\AppData\Local\Temp\ipykernel_19964\4051551321.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['atlas_number'] = metadata['gene_area'].map(region_lookup_dict)


In [59]:
metadata['celltype_idx'] = metadata['celltype_index'].map(lambda x: celltype_to_index.get(x))
print(metadata['celltype_idx'])

0          20
1          56
2          55
3          10
4          71
         ... 
387090    100
387091     30
387092     34
387093     17
387094     44
Name: celltype_idx, Length: 387095, dtype: int64


In [60]:
metadata['atlas_idx'] = metadata['atlas_number'].map(lambda x: atlas_to_index.get(x))
print(metadata['atlas_idx'])

0          96
1          86
2         128
3          69
4          42
         ... 
387090     42
387091     42
387092    132
387093     33
387094     42
Name: atlas_idx, Length: 387095, dtype: int64


In [61]:
print(metadata.shape)
metadata = metadata.dropna(subset=['celltype_idx', 'atlas_idx'])
print(metadata.shape)

(387095, 6)
(387095, 6)


# Full Code

In [ ]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from glob import glob

numpy2ri.activate()  # enables automatic conversion between R and numpy objects

# Import the R function for reading RDS files.
readRDS = ro.r['readRDS']

# Get list of .rds files
file_list = glob(r"D:\Macaque\cellFiles\*.rds")

def load_rds_as_sparse(file_path):
    """
    Load an .rds file that contains a dgCMatrix and return:
      - a scipy.sparse.csc_matrix for the gene expression data,
      - the gene symbols (row names),
      - the cell metadata as a pandas DataFrame.
      
    This function assumes that:
      - The dgCMatrix has slots "i", "p", "x", "Dim", and "Dimnames".
      - The cell metadata is stored in the slot "metadata".
    """
    r_obj = readRDS(file_path)
    
    # Access the S4 slots using ro.r['slot']
    i_slot = np.array(ro.r['slot'](r_obj, 'i'))
    p_slot = np.array(ro.r['slot'](r_obj, 'p'))
    x_slot = np.array(ro.r['slot'](r_obj, 'x'))
    dim = np.array(ro.r['slot'](r_obj, 'Dim'))
    sparse_mat = csc_matrix((x_slot, i_slot, p_slot), shape=(dim[0], dim[1]))
    
    # Get gene names from the first element of the Dimnames slot
    dimnames = ro.r['slot'](r_obj, 'Dimnames')
    genes = list(dimnames[0])
    
    # Convert the metadata (cell info) to a pandas DataFrame.
    meta_r = ro.r['slot'](r_obj, 'metadata')
    metadata = pd.DataFrame(np.array(meta_r))
    
    return sparse_mat, genes, metadata

# Load the region lookup CSV as before.
region_lookup = pd.read_csv('global_region_id_to_atlas_number.csv')
region_lookup_dict = pd.Series(region_lookup.atlas_number.values, index=region_lookup.global_region_id).to_dict()

# Load the per-slide cell type mapping from pickle.
with open("CellID_Per_Slide_Mapping.pkl", "rb") as f:
    slide_lookup = pickle.load(f)

# # --- FIRST PASS: Determine the union of all unique genes ---
# all_genes_set = set()
# for file_path in file_list:
#     _, genes, _ = load_rds_as_sparse(file_path)
#     all_genes_set.update(genes)
# all_genes = sorted(all_genes_set)
# n_genes = len(all_genes)
# print(f"Total unique genes: {n_genes}")
all_genes = pd.read_csv(r"D:\Macaque\macaque_genes.csv")
all_genes = all_genes['gene_name'].to_list()
n_genes = len(all_genes)

# --- Define global atlas dimensions ---
unique_atlas = sorted(region_lookup.atlas_number.unique())
n_atlas = len(unique_atlas)

# To determine global cell type dimension, we need to scan through all slide lookup mappings.
all_celltypes = set()
for key, df in slide_lookup.items():
    all_celltypes.update(df['celltype_index'].unique())
all_celltype = sorted(all_celltypes)
n_celltype = len(all_celltype)

# Create mappings from atlas number / celltype to indices in aggregated arrays.
atlas_to_index = {atlas: idx for idx, atlas in enumerate(unique_atlas)}
celltype_to_index = {ct: idx for idx, ct in enumerate(all_celltype)} # cell type is just the same

# Create a mapping from gene symbol to its global index.
gene_to_global_idx = {gene: idx for idx, gene in enumerate(all_genes)}

# Initialize global cumulative arrays.
cumulative_sum = np.zeros((n_genes, n_atlas, n_celltype), dtype=np.float64)
cumulative_count = np.zeros((n_genes, n_atlas, n_celltype), dtype=np.int64)
print('Initialized global cumulative arrays.')


# --- SECOND PASS: Process each file and update global cumulative arrays ---
for file_path in file_list:
    print(f"Processing {file_path}...")
    sparse_mat, local_genes, metadata = load_rds_as_sparse(file_path)

    # Because of R conversion need to transpose metadata and add column names
    metadata_T = metadata.transpose()
    metadata_T.columns = ['cell_id', 'gene_area','x','y','ry','rx']
    metadata = metadata_T[['cell_id', 'gene_area',]]

    # Extract slide key from file name.
    # For example, if filename is "total_gene_T25.rds", extract "T25".
    file_name = os.path.basename(file_path)
    match = re.search(r"T\d+", file_name)
    if not match:
        print(f"Could not extract slide key from {file_name}. Skipping.")
        continue
    slide_key = match.group(0)
    
    # Get the file-specific cell type mapping dataframe.
    if slide_key not in slide_lookup:
        print(f"Slide key {slide_key} not found in slide_lookup. Skipping {file_name}.")
        continue
    file_mapping_df = slide_lookup[slide_key]
    # Create a dictionary mapping from cell_id to celltype_index for this slide.
    file_cell_lookup = dict(zip(file_mapping_df['cell_id'], file_mapping_df['celltype_index']))

    # Map each cell's cell_id to its celltype_index using the per-file mapping.
    metadata['celltype_index'] = metadata['cell_id'].map(file_cell_lookup)

    # Map each cell's 'gene_area' (global_region_id) to its atlas_number using region_lookup.
    metadata['atlas_number'] = metadata['gene_area'].map(region_lookup_dict)
    
    # Convert the lookup values to indices in our global arrays.
    # To know where in the global array these values should go (can't use atlas number 981 for index 981)
    metadata['celltype_idx'] = metadata['celltype_index'].map(lambda x: celltype_to_index.get(x))
    metadata['atlas_idx'] = metadata['atlas_number'].map(lambda x: atlas_to_index.get(x))
    # Now metadata has two new columns showing indexs in global array for the celltype and atlas
    
    # Group cells by their (atlas_idx, celltype_idx)
    # .indices returns a dict with (atlas_idx, celltype_idx) and list indicies in metadata that
    # correpsond to that combination (all cells of that celltype found in that atlas region in this file)
    groups = metadata.groupby(['atlas_idx', 'celltype_idx']).indices 
    for (atlas_idx, celltype_idx), cell_indices in groups.items():
        # Sum expression over the cells in this group.
        # sparse_mat[:, cell_indices] has shape (n_local_genes, n_group_cells)
        # sparse_mat[:, cell_indices] selects all cells found in this (atlas_idx, celltype_idx) 
        # and returns all their genes
        # The .sum(axis=1) collects the total gene expression for each gene (axis=1 to sum across cells)
        # across all cells for this (atlas_idx, celltype_idx) group
        group_sum = sparse_mat[:, cell_indices].sum(axis=1) 
        # group_sum is a matrix with shape (n_local_genes, 1) because it was a sparse matrix
        group_sum = np.array(group_sum).flatten()  # convert to 1D array so (n_local_genes,)
        group_cell_count = len(cell_indices)
        
        # For each gene in this file (local order), update the corresponding global entry.
        # Local genes is a list of gene names in this file
        for local_idx, gene in enumerate(local_genes):
            global_idx = gene_to_global_idx[gene]
            # There are n genes in this file so use local_idx to go through each group sum
            # So this will only go through each gene (global_idx)
            cumulative_sum[global_idx, atlas_idx, celltype_idx] += group_sum[local_idx]
            cumulative_count[global_idx, atlas_idx, celltype_idx] += group_cell_count

# --- Compute overall mean expression ---
mean_array = np.full((n_genes, n_atlas, n_celltype), np.nan, dtype=np.float64)
valid = cumulative_count > 0 # Can't divide by 0 (no cells found here)
mean_array[valid] = cumulative_sum[valid] / cumulative_count[valid]

# --- Save the aggregated data ---
np.savez('new_aggregated_data.npz',
         mean=mean_array,
         sum=cumulative_sum,
         count=cumulative_count,
         genes=all_genes,
         atlas_numbers=unique_atlas,
         celltype_indices=all_celltype)
print("Aggregated data saved to new_aggregated_data.npz")


{100: 348, 101: 348, 102: 348, 103: 348, 104: 348, 105: 348, 111: 312, 110: 312, 109: 312, 108: 312, 107: 312, 106: 312, 112: 345, 113: 345, 114: 345, 115: 345, 116: 345, 117: 345, 118: 327, 119: 327, 120: 327, 121: 327, 122: 327, 123: 327, 124: 301, 125: 301, 126: 301, 129: 301, 128: 301, 127: 301, 130: 412, 131: 412, 132: 412, 133: 412, 134: 412, 135: 412, 141: 431, 140: 431, 139: 431, 138: 431, 137: 431, 136: 431, 142: 353, 143: 353, 144: 353, 145: 353, 146: 353, 147: 353, 153: 448, 152: 448, 151: 448, 150: 448, 149: 448, 148: 448, 159: 405, 158: 405, 157: 405, 156: 405, 155: 405, 154: 405, 160: 367, 161: 367, 162: 367, 163: 367, 164: 367, 165: 306, 166: 306, 167: 306, 168: 306, 169: 306, 174: 316, 173: 316, 172: 316, 171: 316, 170: 316, 175: 363, 176: 363, 177: 363, 178: 363, 179: 363, 180: 486, 181: 486, 182: 486, 183: 486, 184: 486, 185: 303, 186: 303, 187: 303, 188: 303, 189: 303, 190: 325, 191: 325, 192: 325, 193: 325, 194: 325, 195: 325, 201: 362, 200: 362, 199: 362, 198: 362,